In [30]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from IPython.core.pylabtools import figsize
from IPython.display import IFrame
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
from sklearn import datasets
from sklearn import metrics
from sklearn import neighbors                           
from sklearn import tree, linear_model,neighbors   
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.datasets import load_boston
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import*
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#Preparing for data, I have saved the prepared data. Please ignore this chunk.
sales_with_category = pd.read_csv("sales_with_category.csv")
sales_with_category.info()
sales_category_list_cust_date_NY = sales_with_category[sales_with_category['StateProvince']=='NY'].groupby(['CustomerName','Sale Date','Base.Implement','EquipmentCategoryDesc']).size()
cust_purchase_history_date = sales_category_list_cust_date_NY.to_frame().unstack().unstack()
cust_purchase_history_date1 = cust_purchase_history_date.dropna(axis = 1,how = 'all').sort_index(level = 2,axis = 1)
cust_purchase_history_date1.info()
cust_purchase_history_date1
cust_purchase_history_date1.to_csv('associate_rule_with_date&category_NY.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (70,77,80,84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55377 entries, 0 to 55376
Columns: 107 entries, Unnamed: 0 to sold_to_code_y
dtypes: float64(20), int64(2), object(85)
memory usage: 45.2+ MB
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19190 entries, ('- FIRST BAPTIST WYOMING', '03/20/2019') to ('zimmerman', '08/12/2019')
Data columns (total 39 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   (0, 4WD Loaders, Base)                           2 non-null      float64
 1   (0, AG Tractors, Base)                           3313 non-null   float64
 2   (0, Combine Harvesting, Base)                    111 non-null    float64
 3   (0, Commercial Mowing, Base)                     15 non-null     float64
 4   (0, Compact 4WD Loaders, Base)                   29 non-null     float64
 5   (0, Compact Excavators, Base)                    193 non-null    float64
 6   (0, Comp

In [31]:
#start here.
cust_purchase_history_date2 = pd.read_csv('associate_rule_with_date&category_NY.csv',index_col=0)
cust_purchase_history_date2
associate_rule_cust_date = cust_purchase_history_date2.iloc[1:,0:]
cust_purchase_history_date3 = cust_purchase_history_date2.iloc[1:,0:]
cust_purchase_history_date3
associate_rule_cust_date.info()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Index: 19190 entries, 3/20/19 to 8/12/19
Data columns (total 39 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   4WD Loaders                      2 non-null      object
 1   AG Tractors                      3313 non-null   object
 2   Combine Harvesting               111 non-null    object
 3   Commercial Mowing                15 non-null     object
 4   Compact 4WD Loaders              29 non-null     object
 5   Compact Excavators               193 non-null    object
 6   Compact Track Loaders            121 non-null    object
 7   Crawlers                         2 non-null      object
 8   Golf Products                    1 non-null      object
 9   Harvesters                       3 non-null      object
 10  Hay & Forage                     355 non-null    object
 11  ISG                              122 non-null    object
 12  Irrigation                   

In [32]:
for i in range(cust_purchase_history_date3.shape[0]):
    for j in range(cust_purchase_history_date3.shape[1]):
        if pd.isna(cust_purchase_history_date3.iloc[i][j]):
            associate_rule_cust_date.iloc[i][j] = 0
        else:
            associate_rule_cust_date.iloc[i][j] = 1


In [33]:
new_df_cust_date = associate_rule_cust_date
new_df_cust_date['Base'] = 0
new_df_cust_date['Implement'] = 0
new_df_cust_date = new_df_cust_date.astype('float') 
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19190 entries, 3/20/19 to 8/12/19
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   4WD Loaders                      19190 non-null  float64
 1   AG Tractors                      19190 non-null  float64
 2   Combine Harvesting               19190 non-null  float64
 3   Commercial Mowing                19190 non-null  float64
 4   Compact 4WD Loaders              19190 non-null  float64
 5   Compact Excavators               19190 non-null  float64
 6   Compact Track Loaders            19190 non-null  float64
 7   Crawlers                         19190 non-null  float64
 8   Golf Products                    19190 non-null  float64
 9   Harvesters                       19190 non-null  float64
 10  Hay & Forage                     19190 non-null  float64
 11  ISG                              19190 non-null  float64
 12  Irrigation     

In [8]:
#new_df_cust_date.iloc[0:,25:]

,Agriculture Attachments,Agriculture Implements,Backhoes,Commercial Worksite Attachments,Compact Attachments,Construction Tools,Cutters & Shredders,Landscape Equipment,Material Handling,Snow Equipment,Trailers,Turf Attachments,Turf implements,Worksite Pro Implements,Base,Implement
Sale Date,,,,,,,,,,,,,,,,
2019-03-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2019-11-20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2020-05-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2020-05-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2020-06-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2019-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2021-06-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [34]:
m = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,:25]) == 0):
        m.append(0)
    else:
        m.append(1)
n = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,25:]) == 0):
        n.append(0)
    else:
        n.append(1)
new_df_cust_date['Base'] = m
new_df_cust_date['Implement'] = n
new_df_cust_date.info()
new_df_cust_date.index = pd.to_datetime(new_df_cust_date.index)

<class 'pandas.core.frame.DataFrame'>
Index: 19190 entries, 3/20/19 to 8/12/19
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   4WD Loaders                      19190 non-null  float64
 1   AG Tractors                      19190 non-null  float64
 2   Combine Harvesting               19190 non-null  float64
 3   Commercial Mowing                19190 non-null  float64
 4   Compact 4WD Loaders              19190 non-null  float64
 5   Compact Excavators               19190 non-null  float64
 6   Compact Track Loaders            19190 non-null  float64
 7   Crawlers                         19190 non-null  float64
 8   Golf Products                    19190 non-null  float64
 9   Harvesters                       19190 non-null  float64
 10  Hay & Forage                     19190 non-null  float64
 11  ISG                              19190 non-null  float64
 12  Irrigation     

In [35]:
NY_baseonly = new_df_cust_date.iloc[0:,25:40]
support_NY_baseonly = apriori(NY_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_NY = apriori(new_df_cust_date.iloc[0:,:39], min_support = 0.005, use_colnames = True, verbose = 1)
association_NY = association_rules(support_NY, metric = "confidence", min_threshold = 0.1)

Processing 42 combinations | Sampling itemset size 2
Processing 48 combinations | Sampling itemset size 32


In [16]:
association_NY

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.034028,0.172642,0.018968,0.557427,3.228804,0.013094,1.869428
1,(AG Tractors),(Agriculture Implements),0.172642,0.034028,0.018968,0.109870,3.228804,0.013094,1.085203
2,(Cutters & Shredders),(AG Tractors),0.031527,0.172642,0.009588,0.304132,1.761635,0.004145,1.188959
3,(Landscape Equipment),(AG Tractors),0.043721,0.172642,0.010526,0.240763,1.394578,0.002978,1.089723
4,(AG Tractors),(Material Handling),0.172642,0.062324,0.021834,0.126471,2.029254,0.011075,1.073435
5,(Material Handling),(AG Tractors),0.062324,0.172642,0.021834,0.350334,2.029254,0.011075,1.273514
6,(Snow Equipment),(AG Tractors),0.063210,0.172642,0.010266,0.162407,0.940717,-0.000647,0.987781
7,(Turf implements),(AG Tractors),0.081918,0.172642,0.008702,0.106234,0.615343,-0.005440,0.925699
8,(Snow Equipment),(Mowers),0.063210,0.480771,0.013497,0.213520,0.444120,-0.016893,0.660193
9,(Turf implements),(Mowers),0.081918,0.480771,0.023919,0.291985,0.607326,-0.015465,0.733358


In [36]:
Jan1 = new_df_cust_date.loc['2019-01']
Jan2 = new_df_cust_date.loc['2020-01']
Jan3 = new_df_cust_date.loc['2021-01']
Jan = pd.concat([Jan1,Jan2,Jan3],axis=0)


Feb1 = new_df_cust_date.loc['2019-02']
Feb2 = new_df_cust_date.loc['2020-02']
Feb3 = new_df_cust_date.loc['2021-02']
Feb = pd.concat([Feb1,Feb2,Feb3],axis=0)

Mar1 = new_df_cust_date.loc['2019-03']
Mar2 = new_df_cust_date.loc['2020-03']
Mar3 = new_df_cust_date.loc['2021-03']
Mar = pd.concat([Mar1,Mar2,Mar3],axis=0)

Apr1 = new_df_cust_date.loc['2019-04']
Apr2 = new_df_cust_date.loc['2020-04']
Apr3 = new_df_cust_date.loc['2021-04']
Apr = pd.concat([Apr1,Apr2,Apr3],axis=0)


May1 = new_df_cust_date.loc['2019-05']
May2 = new_df_cust_date.loc['2020-05']
May3 = new_df_cust_date.loc['2021-05']
May = pd.concat([May1,May2,May3],axis=0)

Jun1 = new_df_cust_date.loc['2019-06']
Jun2 = new_df_cust_date.loc['2020-06']
Jun3 = new_df_cust_date.loc['2021-06']
Jun = pd.concat([Jun1,Jun2,Jun3],axis=0)

Jul1 = new_df_cust_date.loc['2019-07']
Jul2 = new_df_cust_date.loc['2020-07']
Jul3 = new_df_cust_date.loc['2021-07']
Jul = pd.concat([Jul1,Jul2,Jul3],axis=0)

Aug1 = new_df_cust_date.loc['2019-08']
Aug2 = new_df_cust_date.loc['2020-08']
Aug3 = new_df_cust_date.loc['2021-08']
Aug = pd.concat([Aug1,Aug2,Aug3],axis=0)

Sep1 = new_df_cust_date.loc['2019-09']
Sep2 = new_df_cust_date.loc['2020-09']
Sep3 = new_df_cust_date.loc['2021-09']
Sep = pd.concat([Sep1,Sep2,Sep3],axis=0)

Oct1 = new_df_cust_date.loc['2019-10']
Oct2 = new_df_cust_date.loc['2020-10']
Oct3 = new_df_cust_date.loc['2021-10']
Oct = pd.concat([Oct1,Oct2,Oct3],axis=0)

Nov1 = new_df_cust_date.loc['2019-11']
Nov2 = new_df_cust_date.loc['2020-11']
Nov3 = new_df_cust_date.loc['2021-11']
Nov = pd.concat([Nov1,Nov2,Nov3],axis=0)

Dec1 = new_df_cust_date.loc['2019-12']
Dec2 = new_df_cust_date.loc['2020-12']
Dec3 = new_df_cust_date.loc['2021-12']
Dec = pd.concat([Dec1,Dec2,Dec3],axis=0)

spring = pd.concat([Mar,Apr,May],axis=0)
summer = pd.concat([Jun,Jul,Aug],axis=0)
fall = pd.concat([Sep,Oct,Nov],axis=0)
winter = pd.concat([Dec,Jan,Feb],axis=0)

In [37]:
spring_baseonly = summer.iloc[0:,25:40]
support_spring_baseonly = apriori(spring_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_spring = apriori(spring.iloc[0:,:39], min_support = 0.005, use_colnames = True, verbose = 1)
association_spring = association_rules(support_spring, metric = "confidence", min_threshold = 0.1)

summer_baseonly = summer.iloc[0:,25:40]
support_summer_baseonly = apriori(summer_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_summer = apriori(summer.iloc[0:,:39], min_support = 0.005, use_colnames = True, verbose = 1)
association_summer = association_rules(support_summer, metric = "confidence", min_threshold = 0.1)

fall_baseonly = fall.iloc[0:,25:40]
support_fall_baseonly = apriori(fall_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_fall = apriori(fall.iloc[0:,:39], min_support = 0.005, use_colnames = True, verbose = 1)
association_fall = association_rules(support_fall, metric = "confidence", min_threshold = 0.1)


winter_baseonly = winter.iloc[0:,25:40]
support_winter_baseonly = apriori(winter_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_winter = apriori(winter.iloc[0:,:39], min_support = 0.005, use_colnames = True, verbose = 1)
association_winter = association_rules(support_winter, metric = "confidence", min_threshold = 0.1)

Processing 42 combinations | Sampling itemset size 2
Processing 54 combinations | Sampling itemset size 3
Processing 42 combinations | Sampling itemset size 2
Processing 48 combinations | Sampling itemset size 3
Processing 42 combinations | Sampling itemset size 2
Processing 4 combinations | Sampling itemset size 4
Processing 42 combinations | Sampling itemset size 2
Processing 48 combinations | Sampling itemset size 3


In [26]:
association_spring

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.036501,0.153711,0.021495,0.588889,3.831144,0.015885,2.058541
1,(AG Tractors),(Agriculture Implements),0.153711,0.036501,0.021495,0.139842,3.831144,0.015885,1.120141
2,(Cutters & Shredders),(AG Tractors),0.021090,0.153711,0.008247,0.391026,2.543902,0.005005,1.389696
3,(Landscape Equipment),(AG Tractors),0.043937,0.153711,0.010815,0.246154,1.601407,0.004062,1.122628
4,(AG Tractors),(Material Handling),0.153711,0.054211,0.020954,0.136324,2.514679,0.012622,1.095073
5,(Material Handling),(AG Tractors),0.054211,0.153711,0.020954,0.386534,2.514679,0.012622,1.379520
6,(Snow Equipment),(AG Tractors),0.024199,0.153711,0.006895,0.284916,1.853584,0.003175,1.183482
7,(Turf implements),(AG Tractors),0.080168,0.153711,0.008382,0.104553,0.680193,-0.003941,0.945102
8,(Snow Equipment),(Mowers),0.024199,0.577531,0.007030,0.290503,0.503008,-0.006946,0.595448
9,(Turf implements),(Mowers),0.080168,0.577531,0.022847,0.284992,0.493465,-0.023452,0.590858


In [27]:
association_summer

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.028782,0.158680,0.016426,0.570681,3.596426,0.011858,1.959660
1,(AG Tractors),(Agriculture Implements),0.158680,0.028782,0.016426,0.103514,3.596426,0.011858,1.083360
2,(Cutters & Shredders),(AG Tractors),0.045057,0.158680,0.010398,0.230769,1.454306,0.003248,1.093716
3,(Landscape Equipment),(AG Tractors),0.046564,0.158680,0.010096,0.216828,1.366452,0.002708,1.074248
4,(AG Tractors),(Material Handling),0.158680,0.055756,0.018987,0.119658,2.146085,0.010140,1.072587
5,(Material Handling),(AG Tractors),0.055756,0.158680,0.018987,0.340541,2.146085,0.010140,1.275772
6,(Snow Equipment),(AG Tractors),0.028330,0.158680,0.006781,0.239362,1.508456,0.002286,1.106071
7,(Snow Equipment),(Mowers),0.028330,0.529234,0.011001,0.388298,0.733697,-0.003993,0.769599
8,(Turf implements),(Mowers),0.070223,0.529234,0.023207,0.330472,0.624434,-0.013958,0.703131


In [28]:
association_fall

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.037212,0.211290,0.019868,0.533898,2.526853,0.012005,1.692142
1,(Cutters & Shredders),(AG Tractors),0.034689,0.211290,0.012930,0.372727,1.764057,0.005600,1.257364
2,(Landscape Equipment),(AG Tractors),0.038158,0.211290,0.009461,0.247934,1.173430,0.001398,1.048725
3,(AG Tractors),(Material Handling),0.211290,0.082624,0.029013,0.137313,1.661912,0.011555,1.063395
4,(Material Handling),(AG Tractors),0.082624,0.211290,0.029013,0.351145,1.661912,0.011555,1.215542
5,(Snow Equipment),(AG Tractors),0.155471,0.211290,0.020498,0.131846,0.624005,-0.012351,0.908491
6,(Turf implements),(AG Tractors),0.093977,0.211290,0.010722,0.114094,0.539988,-0.009134,0.890286
7,(Snow Equipment),(Mowers),0.155471,0.321350,0.028382,0.182556,0.568091,-0.021578,0.830210
8,(Turf implements),(Mowers),0.093977,0.321350,0.028698,0.305369,0.950270,-0.001502,0.976994
9,(Agriculture Implements),(Snow Equipment),0.037212,0.155471,0.005676,0.152542,0.981160,-0.000109,0.996544


In [29]:
association_winter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.037261,0.228097,0.016616,0.445946,1.955074,0.008117,1.393191
1,(Cutters & Shredders),(AG Tractors),0.020141,0.228097,0.006546,0.325000,1.424834,0.001952,1.143560
2,(Landscape Equipment),(AG Tractors),0.042296,0.228097,0.012588,0.297619,1.304793,0.002941,1.098981
3,(AG Tractors),(Material Handling),0.228097,0.082075,0.023162,0.101545,1.237232,0.004441,1.021671
4,(Material Handling),(AG Tractors),0.082075,0.228097,0.023162,0.282209,1.237232,0.004441,1.075387
5,(Snow Equipment),(AG Tractors),0.177744,0.228097,0.018127,0.101983,0.447104,-0.022416,0.859564
6,(Turf implements),(AG Tractors),0.108258,0.228097,0.014099,0.130233,0.570953,-0.010595,0.887482
7,(Mowers),(Snow Equipment),0.212991,0.177744,0.022155,0.104019,0.585217,-0.015703,0.917716
8,(Snow Equipment),(Mowers),0.177744,0.212991,0.022155,0.124646,0.585217,-0.015703,0.899075
9,(Mowers),(Turf implements),0.212991,0.108258,0.022659,0.106383,0.982682,-0.000399,0.997902
